In [199]:
import os
import csv
from countries_languages import country_to_language
import pandas as pd
from difflib import get_close_matches
import numpy as np
import re
from transliterate import translit
from unidecode import unidecode
import Levenshtein
import requests
import bs4
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.parse
from datetime import datetime
from functools import reduce
from urllib.parse import urlsplit

import wikipediaapi

import wikipedia

from urllib.parse import urlparse

import difflib

from datetime import datetime

import calendar

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import AssemblyHelpers
#from AssemblyHelpers import find_money_info_from_name


import requests
from bs4 import BeautifulSoup

def load_csv_dataset(file_path):
    return pd.read_csv(file_path)

from fuzzywuzzy import process

def find_close_matches_variable(this_jersey, dataset_nationality, threshold):
    """
    Find close matches of `this_jersey` in `dataset_nationality` using Levenshtein distance.

    Args:
    - this_jersey (str): The string to find close matches for.
    - dataset_nationality (list): List of strings to search for close matches in.
    - threshold (int): Minimum similarity score required for a match (default is 90).

    Returns:
    - List of strings from `dataset_nationality` that are close matches to `this_jersey`.
    """
    close_matches = process.extract(this_jersey, dataset_nationality, limit=None)
    return [match[0] for match in close_matches if match[1] >= threshold]

#threshold match one 
def threshold_player_match(this_jersey, dataset_nationality):
    THRESHOLD_NUM = 89
    #stops when it returns a name.
    #if it doesnt find a match keep lowering the threshold until you find a match
    #but if you get to threshold of like 50 first you stop and just abandon ship    

    # Loop until someone is found or threshold goes below 50
    while THRESHOLD_NUM >= 50:
        matches = find_close_matches_variable(this_jersey, dataset_nationality, THRESHOLD_NUM)
        if matches:
            #print(f"Player is {this_jersey}. Found matches: {matches}. threshold is {THRESHOLD_NUM}")
            return matches, THRESHOLD_NUM
            #break
        else:
            THRESHOLD_NUM -= 1

    # If threshold reaches below 50 without finding any matches
    if THRESHOLD_NUM < 50:
        return [f"No matches found even with the lowest threshold."], this_jersey #jersey was {this_jersey}


def filter_using_date_of_match(candidate_list_of_names_input, nationality_input, matchdate_Input, SALARY_BOOLEAN): 

    FILTERED_NAMES_USING_MATCH_DATE = AssemblyHelpers.multiNameMatchDateLookup(candidate_list_of_names_input, nationality_input, matchdate_Input) 

    #print('in fudom', type(FILTERED_NAMES_USING_MATCH_DATE))
    #1 MATCH
    if isinstance(FILTERED_NAMES_USING_MATCH_DATE, str) or isinstance(FILTERED_NAMES_USING_MATCH_DATE, np.str_): #YELLOW 3
        #print('were here', FILTERED_NAMES_USING_MATCH_DATE)
        #print('were in fudom in the right place')
        #NAME_FOR_SELENIUM_SEARCH = FILTERED_NAMES_USING_MATCH_DATE
        # MONEY_FOUND_ONLINE = useSeleniumToFindMoney(NAME_FOR_SELENIUM_SEARCH, nationality_input, matchdate_Input, SALARY_BOOLEAN)
        # print('made it out of using selenium')
        # money_thisplayer = [MONEY_FOUND_ONLINE, f'Online WAS Lookup Required - salary boolean is {SALARY_BOOLEAN}', True, False]
        money_thisplayer = [0, 'Lookup Required', True, False]
        return money_thisplayer, FILTERED_NAMES_USING_MATCH_DATE
    #MULTIPLE MATCHES
    elif(len(FILTERED_NAMES_USING_MATCH_DATE) >= 2):
        print(FILTERED_NAMES_USING_MATCH_DATE, type(FILTERED_NAMES_USING_MATCH_DATE), type(FILTERED_NAMES_USING_MATCH_DATE) == str)
        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT MULTIPLE RESULTS: {FILTERED_NAMES_USING_MATCH_DATE}', True, False]
        return money_thisplayer, FILTERED_NAMES_USING_MATCH_DATE
    #0 MATCHES
    elif(len(FILTERED_NAMES_USING_MATCH_DATE) == 0):
        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT 0 RESULTS. BEFORE FILTERING, CANDIDATE NAMES WERE {candidate_list_of_names_input}', True, False]
        return money_thisplayer, ''
    

def check_tokens(this_jersey, match):
    jersey_tokens = this_jersey.lower().split()
    match_tokens = match.lower().split()
    
    if all(token in match_tokens for token in jersey_tokens):
        return True
    elif all(token in jersey_tokens for token in match_tokens) and len(jersey_tokens) == len(match_tokens) + 1:
        return True
    elif all(token in match_tokens for token in jersey_tokens) and len(jersey_tokens) > len(match_tokens):
        return True
    else:
        return False

def filter_multi_word_matches_by_jersey_tokens(jersey_tokens, potential_matches):
    filtered_matches = []

    for match in potential_matches:
        # Split each match into tokens
        match_tokens = match.split()

        # Count the number of tokens that start with each character in jersey_tokens
        match_token_start_chars = {token[0] for token in match_tokens}
        #print(match_token_start_chars, f"*{jersey_tokens}")

        #for token in jersey_tokens:
        #   print(token, token[0] in match_token_start_chars)

        # Check if the match contains at least one token for each character in jersey_tokens
        if all(token[0] in match_token_start_chars for token in jersey_tokens):
            filtered_matches.append(match)

    return filtered_matches

def last_name_match(name, dataset_of_names):
    matching_names = []
    for full_name in dataset_of_names:
        
        # Split the full name into tokens
        full_name_tokens = set(re.split(r'\s+|[-–]', full_name))
        
        # Check if any token matches the last name or its unidecoded version
        for token in full_name_tokens:
            #if full_name == 'Fródi Benjaminsen':
                #print(token, name, token == name)
                #print(full_name_tokens)
            
            if token == name or unidecode(token) == name:
                matching_names.append(full_name)
                break  # Once a match is found, move to the next name
    return matching_names

def check_other_team(lookup_row, transfermarkt_data, results_df):
    new_rows = []
    wrong_rows = []

    name = lookup_row['Name']
    original_jersey = lookup_row['ORIGINAL JERSEY']
    season = lookup_row['Season']
    original_country_code = lookup_row['Team Country Code']
    match_id = lookup_row['Match ID']
    opposing_country_code = ""
    game_row = results_df[results_df['Match ID'] == match_id]
    #print(game_row)
    
    if len(game_row) != 1:
        raise ValueError("There should only be one match per ID")

    #print(original_country_code)
    team2_code = game_row.iloc[0]['Team 2 Code']
    team1_code = game_row.iloc[0]['Team 1 Code']

    #print(f"1{team1_code}, 2{team2_code}")
    if original_country_code == 'CG':
        original_country_code = ' CG'
    elif original_country_code  == 'NI':
        original_country_code = ' NI'
    if original_country_code == team1_code:
        opposing_country_code = team2_code
    elif original_country_code == team2_code:
        opposing_country_code = team1_code
    else:
        raise ValueError("Original country code does not match any team in the match")

    dataset_nationality = transfermarkt_data[transfermarkt_data['Team 1 Code'] == opposing_country_code]['Name'].unique()
    possible_names = threshold_player_match(original_jersey, dataset_nationality)
    
    return possible_names[0]

def add_correct_money(money_column):
    if money_column == 0:
        return 0

    if 'm' in money_column:
        value_rough = money_column.split('m')[0].split('€')[1]
        millinum = (float(value_rough) * 1000000)
        return millinum
    elif 'k' in money_column:
        #print(i, 'k')
        value_rough = money_column.split('k')[0].split('€')[1]
        millinum = (float(value_rough) * 1000)

        return millinum
    elif '-' in money_column:
        return 0
    else:
        return 0

def remove_nation(match, nation1):
    # Split the match string by 'vs' and strip any leading or trailing spaces
    parts = [part.strip() for part in match.split('vs')]
    # Remove the nation1 from the list of parts
    parts.remove(nation1)
    # Join the remaining parts and return
    return ''.join(parts)


In [2]:
#loading CSVs

#PLAYER DATA - MARKET VALUE AND SALARY
leagues_salary = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1//CSVs we use often/latest_capology_data_money_fixed.csv')

leagues_value = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/MarketValuesComplete.csv')


leagues_value_large = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/most_updated_transfermarkt_dataset.csv')

#COUNTRY CODES
countries_codes = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/countries_and_codes.csv')

#FIFA RANKINGS
fifa_rankings = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/fifa_ranking-2023-10-26 - fifa_ranking-2023-07-20.csv')

#MATCH DATA - LINEUPS AND RESULTS
# lineups_data = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/newest_lineups_data_march_24.csv')
lineups_data = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/newest_lineups_data_april_29.csv')
results_data = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/newest_results_data_march_24.csv')

/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_48300/813481528.py:9: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  leagues_value_large = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/most_updated_transfermarkt_dataset.csv')


In [3]:
combined_df = load_csv_dataset('combined_df_may_1.csv')
combined_df = combined_df.drop(columns={'Unnamed: 0'})

/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_48300/931677539.py:49: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


## Weird 0 Cases

In [ ]:
pink_group_0_cases = load_csv_dataset('April 25 - CONCACAF & CONMEBOL OUT - DATA CSVs/Pink Group/CONCACAF & CONMEBOL OUT - odd_name_found_0_cases.csv')
pink_group_0_cases = pink_group_0_cases.drop(columns={'Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'To Remove'})

In [ ]:
for index, row in pink_group_0_cases.iterrows():
    if row['Nationality'] == 'Namibia':
        pink_group_0_cases.at[index, 'Team Country Code'] = 'NB'
    elif row['Nationality'] == 'Congo':
        pink_group_0_cases.at[index, 'Nationality'] = 'Republic of the Congo'
    elif row['Nationality'] == 'Republic of the Congo':
        pink_group_0_cases.at[index, 'Team Country Code'] = 'CG'

    

In [ ]:
pink_group_0_cases['Status New'] = ''

pink_group_0_cases['Market Value New'] = 0

In [23]:
def vet_group_using_initials(jersey, names_found, names_same_nation_last_name):
    correct_names_list = []

    jersey_tokens_initials = [token[0] for token in re.split(r'\s+|[-–]', jersey)]

    names_found_tokens_initials = [token[0] for token in re.split(r'\s+|[-–]', names_found)]

    for name in names_same_nation_last_name:
        name_initials = [token[0] for token in re.split(r'\s+|[-–]', name)]
        
        if set(jersey_tokens_initials) == set(name_initials):
            correct_names_list.append(name)
        elif set(jersey_tokens_initials).issubset(set(name_initials)):
            correct_names_list.append(name)
        #else:
            #print('fail')

    if set(jersey_tokens_initials) == set(names_found_tokens_initials):
        correct_names_list.append(names_found)
    elif set(jersey_tokens_initials).issubset(set(names_found_tokens_initials)):
            correct_names_list.append(names_found)


    return correct_names_list

In [ ]:
success_counter = 0
done_counter = 0
for index, row in pink_group_0_cases.iterrows():
    
    if row['Status New'] != 'Done':
        correct_name = ''
        player_this = row['ORIGINAL JERSEY']
        if player_this == 'Král':
            player_this = 'Alex Kral'
        elif player_this == 'Oršić':
            player_this = 'Mislav Orsic'
        elif player_this == 'Višća':
            player_this = 'Edin Visca'
        elif player_this == 'evčík':
            player_this = 'Petr Sevcik'
        nationality = row['Nationality']
        date = row['Date']
        match = row['Match']
        
        full_num = row['Season']
        
        if nationality == 'Republic of the Congo' or nationality == 'Namibia':
            country_code_this = row['Team Country Code']
            dataset_nationality = leagues_value[(leagues_value['Team 1 Code'] == country_code_this) & 
                                        (leagues_value['Season'].isin([full_num, full_num + 1, full_num - 1]))]['Name'].unique()
            
            best_match = threshold_player_match(player_this, dataset_nationality)[0]
            
            if len(best_match) == 1:
                correct_name = best_match[0]
                print(index, nationality, player_this)
                #print(index, best_match)
            else:
                print(index, nationality, player_this)
                #print(index, best_match)

                last_name = re.split(r'\s+|[-–]', player_this)[-1]
            # #TRY TO FIND OTHER PLAYERS FROM THE NATION W THE SAME LAST NAME
                names_same_nation_last_name = last_name_match(last_name, dataset_nationality)

                vetted_list_using_initials = vet_group_using_initials(player_this, player_this, best_match)
                if player_this in vetted_list_using_initials:
                    vetted_list_using_initials.remove(player_this)
                #print(index, vetted_list_using_initials)
                correct_name = vetted_list_using_initials[0]
            

            number, reason = find_in_transfermarkt(correct_name, full_num, country_code_this, leagues_value)
            #print(player_this, reason, add_correct_money(number))

            #add a new row to lookup dataset 1 
            if type(number) == float:
                0==0
            else:
                number = add_correct_money(number)
            print('**')
            print(index, player_this, correct_name, number, reason)
            if reason == 'Found in Curr Season':
                pink_group_0_cases.at[index, 'Name(s) Found'] = correct_name
                pink_group_0_cases.at[index, 'Status New'] = 'Done'
                pink_group_0_cases.at[index, 'Market Value New'] = number
                pink_group_0_cases.at[index, 'Match Case'] = 'single'
                success_counter += 1
                
        else:
            country_code_this = row['Team Country Code']
            dataset_nationality = leagues_value[(leagues_value['Team 1 Code'] == country_code_this) & 
                                        (leagues_value['Season'].isin([full_num, full_num + 1, full_num - 1]))]['Name'].unique()
            
            best_match = threshold_player_match(player_this, dataset_nationality)[0]
            
            if len(best_match) >= 2:
                print(index, best_match)
                vetted_list_using_initials = vet_group_using_initials(player_this, player_this, best_match)
                if player_this in vetted_list_using_initials:
                    vetted_list_using_initials.remove(player_this)
                
                for name in vetted_list_using_initials:
                    if player_this in unidecode(name):
                        0==0
                    elif unidecode(player_this) in name:
                        0==0
                    else:
                        vetted_list_using_initials.remove(name)
                if len(vetted_list_using_initials) == 1:
                    correct_name = vetted_list_using_initials[0]
                #print(index, nationality, player_this, vetted_list_using_initials)

                number, reason = find_in_transfermarkt(correct_name, full_num, country_code_this, leagues_value)
                #print(player_this, reason, add_correct_money(number))

                #add a new row to lookup dataset 1 
                if type(number) == float:
                    0==0
                else:
                    number = add_correct_money(number)
                print('**')
                print(index, player_this, correct_name, number, reason)
                if reason == 'Found in Curr Season':
                    pink_group_0_cases.at[index, 'Name(s) Found'] = correct_name
                    pink_group_0_cases.at[index, 'Status New'] = 'Done'
                    pink_group_0_cases.at[index, 'Market Value New'] = number
                    pink_group_0_cases.at[index, 'Match Case'] = 'single'
                    success_counter += 1
                
            else:
                #print(index, player_this, best_match)
                correct_name = best_match[0]
                number, reason = find_in_transfermarkt(correct_name, full_num, country_code_this, leagues_value)
                #success_counter += 1
                if type(number) == float:
                    0==0
                else:
                    number = add_correct_money(number)
                print('**')
                print(index, player_this, correct_name, number, reason)
                if reason == 'Found in Curr Season':
                    pink_group_0_cases.at[index, 'Name(s) Found'] = correct_name
                    pink_group_0_cases.at[index, 'Status New'] = 'Done'
                    pink_group_0_cases.at[index, 'Market Value New'] = number
                    pink_group_0_cases.at[index, 'Match Case'] = 'single'
                    success_counter += 1


            


                # if len(vetted_list_using_initials) == 1:
                #     correct_name = vetted_list_using_initials[0]
                # elif len(vetted_list_using_initials) >= 2:
                #     initial_double_letters = multiple_occurrences(jersey_tokens_initials)
                #     vetted_list_using_initials = vet_using_repeated_letters(initial_double_letters, vetted_list_using_initials)
    else:
        done_counter += 1
print(success_counter, done_counter)

In [ ]:
for i in range(0, len(pink_group_0_cases)):
    if pink_group_0_cases.at[i, 'Market Value New'] != 0:
        if pink_group_0_cases.at[i, 'Market Value New'] != pink_group_0_cases.at[i, 'Market Value']:
            print(i)
        #print(pink_group_0_cases.at[i, 'Market Value New'], pink_group_0_cases.at[i, 'Market Value'])

In [ ]:
pink_group_0_cases[pink_group_0_cases['Status New'] == 'Done']['Lookup Return Case'].value_counts()

In [ ]:
for i in range(0, len(pink_group_0_cases)):
    if pink_group_0_cases.at[i, 'Market Value'] == pink_group_0_cases.at[i, 'Market Value New']:
        0==0
    else:
        print(i)

In [ ]:
pink_group_0_cases_final = pink_group_0_cases.drop(columns={'Status New', 'Market Value New'})
pink_group_0_cases_final.to_csv('pink_group_weird_0_cases.csv')

## Online Lookup - 0 Names Found one

In [7]:
pink_group_lookup_cases = load_csv_dataset('April 25 - CONCACAF & CONMEBOL OUT - DATA CSVs/Pink Group/pink_group_lookup_cases.csv')
#pink_group_lookup_cases = pink_group_lookup_cases.drop(columns={'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'To Remove'})
#pink_group_lookup_cases['Status New'] = ''
# pink_group_lookup_cases['Market Value New'] = 0


In [277]:
for index, row in pink_group_lookup_cases.iterrows():
    if row['Nationality'] == 'Namibia':
        pink_group_lookup_cases.at[index, 'Team Country Code'] = 'NB'


In [ ]:
for index,row in pink_group_lookup_cases.iterrows():
    if 'Olympics' in row['Competition']:
        pink_group_lookup_cases.at[index, 'Match Case'] = 'Olympic'
    elif 'U-20' in row['Competition']:
        pink_group_lookup_cases.at[index, 'Match Case'] = 'U-20'

In [115]:
case_counter = 0
success_counter = 0
done_cases = 0
for index,row in pink_group_lookup_cases.iterrows():
    if row['Match Case'] == 'Olympic' or row['Match Case'] == 'U-20':
        if row['Status New'] != 'Done':
            case_counter += 1
            name_player = row['ORIGINAL JERSEY']
            full_num = row['Season']
            country_code_this = row['Team Country Code']

            dataset_nationality = leagues_value_large[(leagues_value_large['Team 1 Code'] == country_code_this) & 
                (leagues_value['Season'].isin([full_num, full_num + 1, full_num - 1]))]['Name'].unique()
            if name_player in dataset_nationality:
                print(index, name_player, full_num, 'in dataset nationality')
                correct_name = name_player
                #if they are here, add their $ info. 
                number, reason = find_in_transfermarkt(correct_name, full_num, country_code_this, leagues_value_large)
                #success_counter += 1
                if type(number) == float:
                    0==0
                else:
                    number = add_correct_money(number)
                
                if reason == 'Found in Curr Season':
                    print('**')
                    print(index, correct_name, number, reason)
                    pink_group_lookup_cases.at[index, 'Name(s) Found'] = correct_name
                    pink_group_lookup_cases.at[index, 'Status New'] = 'Done'
                    pink_group_lookup_cases.at[index, 'Lookup Return Case'] = reason
                    pink_group_lookup_cases.at[index, 'Market Value New'] = number
                    # pink_group_lookup_cases.at[index, 'Match Case'] = 'single'
                    #success_counter += 1
                else:
                    success_counter += 1
                    print(index, correct_name, number, reason)
                    pink_group_lookup_cases.at[index, 'Name(s) Found'] = correct_name
                    pink_group_lookup_cases.at[index, 'Lookup Return Case'] = 'Did Not Find in Curr Season, One Up or One Down'
                    pink_group_lookup_cases.at[index, 'Status New'] = 'Done'
                    pink_group_lookup_cases.at[index, 'Status'] = 'FAIL - LARGE'
                    #if theyre in the dataset in general but not in the seasons we want:
                        #set status column to FAIL - NEW. and set the Lookup Return case to Did Not Find in Curr Season, One Up or One Down - Use Large Dataset


            elif name_player in leagues_value_large['Name'].unique():
                0==0
                success_counter += 1
                print(index, name_player, full_num, 'in huge dataset')
                pink_group_lookup_cases.at[index, 'Name(s) Found'] = correct_name
                pink_group_lookup_cases.at[index, 'Lookup Return Case'] = 'Did Not Find in Curr Season, One Up or One Down'
                pink_group_lookup_cases.at[index, 'Status New'] = 'Done'
                pink_group_lookup_cases.at[index, 'Status'] = 'FAIL - LARGE'
                
                #if theyre in the dataset in general but not in the seasons we want:
                    #set status column to FAIL - LARGE. and set the Lookup Return case to Did Not Find in Curr Season, One Up or One Down - Use Large Dataset

            else:
                0==0
                success_counter += 1
                
                correct_name = threshold_player_match(name_player, dataset_nationality)[0][0]
                number, reason = find_in_transfermarkt(correct_name, full_num, country_code_this, leagues_value_large)
                print(index,name_player, correct_name, number, reason)
                pink_group_lookup_cases.at[index, 'Name(s) Found'] = correct_name
                pink_group_lookup_cases.at[index, 'Status New'] = 'Done'
                pink_group_lookup_cases.at[index, 'Lookup Return Case'] = reason
                pink_group_lookup_cases.at[index, 'Market Value New'] = number
                
        else:
            done_cases += 1
    elif row['Nationality'] == 'Namibia' or row['Nationality'] == 'Republic of the Congo':
        if row['Status New'] != 'Done':
            
            
            case_counter += 1
            correct_name = ''
            name_player = row['ORIGINAL JERSEY']
            full_num = row['Season']
            country_code_this = row['Team Country Code']

            dataset_nationality = leagues_value[(leagues_value['Team 1 Code'] == country_code_this) & 
                (leagues_value['Season'].isin([full_num, full_num + 1, full_num - 1]))]['Name'].unique()
            best_match = threshold_player_match(name_player, dataset_nationality)[0]
            
            if len(best_match) == 1:
                correct_name = best_match[0]
                #print(index, name_player, correct_name)
                #print(index, best_match)
            else:
                #print(index, nationality, name_player)
                #print(index, best_match)

                
            # #TRY TO FIND OTHER PLAYERS FROM THE NATION W THE SAME LAST NAME
                last_name = re.split(r'\s+|[-–]', name_player)[-1]
                names_same_nation_last_name = last_name_match(last_name, dataset_nationality)

                vetted_list_using_initials = vet_group_using_initials(name_player, name_player, best_match)
                if name_player in vetted_list_using_initials:
                    vetted_list_using_initials.remove(name_player)
                #print(index, vetted_list_using_initials)
                correct_name = vetted_list_using_initials[0]
                #print(index, name_player, correct_name)
            if correct_name in dataset_nationality:
                success_counter += 1
                print(index, name_player)
                #print(index, name_player, correct_name, full_num, '***in dataset nationality')
                number, reason = find_in_transfermarkt(correct_name, full_num, country_code_this, leagues_value)
                if type(number) == float:
                    0==0
                else:
                    number = add_correct_money(number)

                pink_group_lookup_cases.at[index, 'Name(s) Found'] = correct_name
                pink_group_lookup_cases.at[index, 'Lookup Return Case'] = reason
                pink_group_lookup_cases.at[index, 'Market Value New'] = number
                pink_group_lookup_cases.at[index, 'Status New'] = 'Done'
                pink_group_lookup_cases.at[index, 'Status'] = 'SUCCESS'
            else:
                case_counter += 1
        else:
            done_cases += 1    
    else:
        if row['Status New'] != 'Done':
            case_counter += 1
            correct_name = ''
            name_player = row['ORIGINAL JERSEY']
            full_num = row['Season']
            country_code_this = row['Team Country Code']

            dataset_nationality = leagues_value_large[(leagues_value_large['Team 1 Code'] == country_code_this) & 
                (leagues_value_large['Season'].isin([full_num, full_num + 1, full_num - 1]))]['Name'].unique()
            best_match = threshold_player_match(name_player, dataset_nationality)[0]
            
            if len(best_match) == 1:
                vetted_list_using_initials = vet_group_using_initials(name_player, name_player, best_match)
                if name_player in vetted_list_using_initials:
                    vetted_list_using_initials.remove(name_player)
                if len(vetted_list_using_initials) == 1:
                    success_counter += 1
                    
                    correct_name = vetted_list_using_initials[0]

                    number, reason = find_in_transfermarkt(correct_name, full_num, country_code_this, leagues_value)
                    if type(number) == float:
                        0==0
                    else:
                        number = add_correct_money(number)
                    print(index, 'derp', name_player, vetted_list_using_initials, number, reason)

                    pink_group_lookup_cases.at[index, 'Name(s) Found'] = correct_name
                    pink_group_lookup_cases.at[index, 'Lookup Return Case'] = reason
                    pink_group_lookup_cases.at[index, 'Market Value New'] = number
                    pink_group_lookup_cases.at[index, 'Status New'] = 'Done'
                    pink_group_lookup_cases.at[index, 'Status'] = 'SUCCESS'
                else:
                    print(index, full_num, name_player, best_match)


                # last_name = re.split(r'\s+|[-–]', name_player)[-1]
                # names_same_nation_last_name = last_name_match(last_name, dataset_nationality)

                # vetted_list_using_initials = vet_group_using_initials(name_player, name_player, names_same_nation_last_name)
                
            else:
                
                vetted_list_using_initials = vet_group_using_initials(name_player, name_player, best_match)
                if name_player in vetted_list_using_initials:
                    vetted_list_using_initials.remove(name_player)
                if vetted_list_using_initials == []:
                    print(index, 'hmm', name_player)
                else:
                    success_counter += 1
                    
                    correct_name = vetted_list_using_initials[0]

                    number, reason = find_in_transfermarkt(correct_name, full_num, country_code_this, leagues_value)
                    if type(number) == float:
                        0==0
                    else:
                        number = add_correct_money(number)

                    print(index, 'hi', full_num, name_player, vetted_list_using_initials, number, reason)
                    pink_group_lookup_cases.at[index, 'Name(s) Found'] = correct_name
                    pink_group_lookup_cases.at[index, 'Lookup Return Case'] = reason
                    pink_group_lookup_cases.at[index, 'Market Value New'] = number
                    pink_group_lookup_cases.at[index, 'Status New'] = 'Done'
                    pink_group_lookup_cases.at[index, 'Status'] = 'SUCCESS'
            
        else:
            done_cases += 1    
            
        


print(case_counter, success_counter, done_cases)

        #for all these people, andthe U-20 guy.

    

89 2019 Henok Goitom ['No matches found even with the lowest threshold.']
155 2019 Henok Goitom ['No matches found even with the lowest threshold.']
2 0 222


In [ ]:
#guys to search for 

#A Gouiri
#H Goitom

In [140]:
for index, row in pink_group_lookup_cases.iterrows():
    if row['ORIGINAL JERSEY'] == 'Henok Goitom':
        pink_group_lookup_cases.at[index, 'Market Value New'] = 1200000.0
        pink_group_lookup_cases.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
    if row['ORIGINAL JERSEY'] == 'N Tella':
        pink_group_lookup_cases.at[index, 'Market Value New'] = 23000000.0
        pink_group_lookup_cases.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
        pink_group_lookup_cases.at[index, 'Match Case'] = 'single'
    if row['ORIGINAL JERSEY'] == 'F Koné':
        pink_group_lookup_cases.at[index, 'Market Value New'] = 200000.0
        pink_group_lookup_cases.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
        pink_group_lookup_cases.at[index, 'Match Case'] = 'single'
        pink_group_lookup_cases.at[index, 'Name'] = 'Francis Koné'
    if row['ORIGINAL JERSEY'] == 'A Gouiri':
        pink_group_lookup_cases.at[index, 'Market Value New'] = 35000000.0
        pink_group_lookup_cases.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
        pink_group_lookup_cases.at[index, 'Match Case'] = 'single'
        pink_group_lookup_cases.at[index, 'Name'] = 'Amine Gouiri'
    if row['ORIGINAL JERSEY'] == 'Henok Goitom' and row['Name'] == 0:
        pink_group_lookup_cases.at[index, 'Market Value New'] = 600000.0
        pink_group_lookup_cases.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
        pink_group_lookup_cases.at[index, 'Status New'] = 'Done'
        pink_group_lookup_cases.at[index, 'Name(s) Found'] = 'Henok Goitom'
    
    

In [109]:
for index, row in pink_group_lookup_cases.iterrows():
    if row['ORIGINAL JERSEY'] == 'H Goitom':
        pink_group_lookup_cases.at[index, 'ORIGINAL JERSEY'] = 'Henok Goitom'
    

In [74]:
for index, row in pink_group_lookup_cases.iterrows():
    if row['Nationality'] == 'Guadeloupe' or row['Nationality'] == 'Martinique':
        pink_group_lookup_cases.at[index, 'Status New'] = 'Done'
        pink_group_lookup_cases.at[index, 'Status'] = 'FAIL - NO DATA'
    if row['Nationality'] == 'Congo':
        pink_group_lookup_cases.at[index, 'Team Country Code'] = 'CG'


In [152]:
for index, row in pink_group_lookup_cases.iterrows():
    pink_group_lookup_cases.at[index, 'Name'] = pink_group_lookup_cases.at[index, 'Name(s) Found']
    if row['Market Value New'] != 0:
        #print(row['Market Value New'], row['Market Value'])
        pink_group_lookup_cases.at[index, 'Market Value'] = pink_group_lookup_cases.at[index, 'Market Value New']
    if row['Status'] == 'FAIL':
        pink_group_lookup_cases.at[index, 'Status'] = 'SUCCESS'

In [154]:
pink_group_lookup_cases['Status'].value_counts()

Status
SUCCESS           130
FAIL - NO DATA     66
FAIL - LARGE       28
Name: count, dtype: int64

In [156]:
pink_group_lookup_cases.to_csv('pink_group_lookup_cases_DONE.csv')

## Cyrillic Group

In [318]:
pink_group_cyrillic = load_csv_dataset('April 25 - CONCACAF & CONMEBOL OUT - DATA CSVs/Pink Group/CONCACAF & CONMEBOL OUT - Cyrillic_Guys_Nobody_Found.csv')
pink_group_cyrillic = pink_group_cyrillic.drop(columns={'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0'})
pink_group_cyrillic = pink_group_cyrillic.rename(columns={'Match(es) Found': 'Name(s) Found'})
pink_group_cyrillic['Status New'] = ''
pink_group_cyrillic['Market Value New'] = 0
pink_group_cyrillic['Lookup Return Case'] = ''

In [319]:
pink_group_cyrillic.at[39, 'Name(s) Found'] = 'Niall McGinn'
pink_group_cyrillic.at[39, 'Status'] = 'SUCCESS'
pink_group_cyrillic.at[39, 'Status New'] = 'Done'
pink_group_cyrillic.at[39, 'Lookup Return Case'] = 'Found in Curr Season'
pink_group_cyrillic.at[39, 'Market Value New'] = 700000
pink_group_cyrillic.at[2, 'Name(s) Found'] = 'Martin Büchel'
pink_group_cyrillic.at[2, 'Status'] = 'SUCCESS'
pink_group_cyrillic.at[2, 'Status New'] = 'Done'
pink_group_cyrillic.at[2, 'Lookup Return Case'] = 'Found in Curr Season'
pink_group_cyrillic.at[2, 'Match Case'] = 'single'
pink_group_cyrillic.at[39, 'Match Case'] = 'single'


/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_48300/3226868144.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Niall McGinn' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  pink_group_cyrillic.at[39, 'Name(s) Found'] = 'Niall McGinn'


In [321]:
case_counter = 0
done_cases = 0
broken_counter = 0
for index,row in pink_group_cyrillic.iterrows():
    if row['Status New'] != 'Done':
        this_found_name = row['English Name']
        if this_found_name == 'L Kasharo':
            this_found_name = 'L Casciaro'
        elif this_found_name == 'F Witayoli':
            this_found_name = 'F Vitaoli'
        elif this_found_name == 'D Simonchini':
            this_found_name = 'D Simoncini'
        elif this_found_name == 'Asimoncini':
            this_found_name = 'A Simoncini'
        if index == 2:
            this_found_name = 'Martin Büchel'
        elif index == 39:
            this_found_name = 'Niall McGinn'
        elif this_found_name == 'E Vishnyakov':
            this_found_name = 'Eduards Visnakovs'
        elif this_found_name == 'A  Vishnyakov':
            this_found_name = 'Aleksejs Visnakovs'
        elif this_found_name == 'R Gurbanov':
            this_found_name = 'Ruslan Qurbanov'
        elif this_found_name == 'Aroyan':
            this_found_name = 'Varazdat Haroyan'
        elif this_found_name == 'Ceyhan Yildiz':
            this_found_name = 'Seyhan Yildiz'
        elif this_found_name == 'Garaev':
            this_found_name = 'Qara Qaraev'
        elif this_found_name == 'Karaev':
            this_found_name = 'Qara Qaraev'
        elif this_found_name == 'Dossajunior':
            this_found_name = 'Dossa Júnior'
        elif this_found_name == 'Armash':
            this_found_name = 'Igor Armaş'
        elif this_found_name == 'Ehri':
            this_found_name = 'Yves Oehri'
        elif this_found_name == 'Vieser':
            this_found_name = 'Sandro Wieser'
        elif this_found_name == 'Askovski':
            this_found_name = 'Stefan Ashkovski'
        elif this_found_name == 'Zhans':
            this_found_name = 'Laurent Jans'
        elif this_found_name == 'Grośowski':
            this_found_name = 'Patrik Hrosovsky'
        elif this_found_name == 'Tsaunya':
            this_found_name = 'Aleksandrs Cauna'
        elif this_found_name == 'Kazaishvili':
            this_found_name = 'Valeri Qazaishvili'
        elif this_found_name == 'Diomber':
            this_found_name == 'Norbert Gyomber'
        elif this_found_name == 'Colsen':
            this_found_name = 'Klaemint Olsen'
        elif this_found_name == 'Siquero':
            this_found_name = 'Gabriel Cichero'
        elif this_found_name == 'Kipchu':
            this_found_name = 'Alexandru Chipciu'
        elif this_found_name == 'Khalsti':
            this_found_name = 'Markus Halsti'
        elif this_found_name == 'Yuhas':
            this_found_name = 'Roland Juhász'
        elif this_found_name == 'Ogani':
            this_found_name = 'Orel Dgani'
        elif this_found_name == 'Mikhailovmmm':
            this_found_name = 'Maxim Mihaliov'

            



        nationality = row['Nationality']
        
        found_names_this_country = combined_df[combined_df['ORIGINAL JERSEY'] == this_found_name]['Name(s) Found'].unique()
        if len(found_names_this_country) == 1:
            case_counter += 1
            #print(index, this_found_name, found_names_this_country)
            full_num = row['Season']
            country_code_this = row['Country Code']

            correct_name = found_names_this_country[0]

            number, reason = find_in_transfermarkt(correct_name, full_num, country_code_this, leagues_value)
            if type(number) == float:
                0==0
            else:
                number = add_correct_money(number)
            

            if reason != 'Did Not Find in Curr Season, One Up, or One Down':
                print(index, this_found_name, found_names_this_country, nationality, full_num, number, reason)
                pink_group_cyrillic.at[index, 'Name(s) Found'] = correct_name
                pink_group_cyrillic.at[index, 'Lookup Return Case'] = reason
                pink_group_cyrillic.at[index, 'Market Value New'] = number
                pink_group_cyrillic.at[index, 'Status New'] = 'Done'
                pink_group_cyrillic.at[index, 'Status'] = 'SUCCESS'
                pink_group_cyrillic.at[index, 'Match Case'] = 'single'
            else:
                if index == 146:
                    print(index, 'did not work', full_num, number, reason)
                else:
                    print(index, 'did not work', full_num, number, reason)
        else:
            if len(found_names_this_country) == 0:
                if this_found_name == '[]':
                    broken_counter += 1
                else:
                    
                    found_names_this_country = combined_df[combined_df['Nationality'] == nationality]['Name(s) Found'].unique()
                    best_match_existing_data = threshold_player_match(this_found_name, found_names_this_country)[0]
                    if len(best_match_existing_data) == 1:
                        if best_match_existing_data == ['No matches found even with the lowest threshold.']:
                            0==0
                        else:
                            
                            vetted_group = vet_group_using_initials(this_found_name, this_found_name, best_match_existing_data)
                            if this_found_name in vetted_group:
                                vetted_group.remove(this_found_name)
                            if len(vetted_group) != 0:
                                case_counter += 1
                                correct_name = vetted_group[0]
                                print(index, this_found_name, row['ORIGINAL JERSEY'], correct_name, row['Nationality'], row['Match'], row['Date'])
                                full_num = row['Season']
                                country_code_this = row['Country Code']

                                number, reason = find_in_transfermarkt(correct_name, full_num, country_code_this, leagues_value)
                                #print(player_this, reason, add_correct_money(number))

                                #add a new row to lookup dataset 1 
                                if type(number) == float:
                                    0==0
                                else:
                                    number = add_correct_money(number)
                                print('**')
                                print(index, correct_name, number, reason)
                                # if reason == 'Found in Curr Season':
                                pink_group_cyrillic.at[index, 'Name(s) Found'] = correct_name
                                pink_group_cyrillic.at[index, 'Status New'] = 'Done'
                                pink_group_cyrillic.at[index, 'Market Value New'] = number
                                pink_group_cyrillic.at[index, 'Match Case'] = 'single'
                                pink_group_cyrillic.at[index, 'Lookup Return Case'] = reason
                                success_counter += 1
                                
            else:
                print(index, 'yo', this_found_name, row['Date'], row['Match'], found_names_this_country)
                
    else:
        done_cases += 1
            
print(case_counter, broken_counter, success_counter, done_cases)
    

146 did not work 2014 0 Did Not Find in Curr Season, One Up, or One Down
1 3 172 66


In [344]:
for index, row in pink_group_cyrillic.iterrows():
    
    if row['ORIGINAL JERSEY'] == '2. Шано':
        pink_group_cyrillic.at[index, 'ORIGINAL JERSEY'] = 'Maxime Chanot'
        pink_group_cyrillic.at[index, 'Name(s) Found'] = 'Maxime Chanot'
    elif row['ORIGINAL JERSEY'] == '1. Пачовски':
        pink_group_cyrillic.at[index, 'ORIGINAL JERSEY'] = 'Tome Pacovski'
        pink_group_cyrillic.at[index, 'Name(s) Found'] = 'Tome Pacovski'
    elif row['ORIGINAL JERSEY'] == '2. Нес':
        pink_group_cyrillic.at[index, 'ORIGINAL JERSEY'] = 'Jónas Tór Naes'
        pink_group_cyrillic.at[index, 'Name(s) Found'] = 'Jónas Tór Naes'
    elif row['ORIGINAL JERSEY'] == '23. Логвиненко':
        pink_group_cyrillic.at[index, 'ORIGINAL JERSEY'] = 'Yuriy Logvinenko'
        pink_group_cyrillic.at[index, 'Name(s) Found'] = 'Yuriy Logvinenko'
    elif row['ORIGINAL JERSEY'] == '5. Дьомбер':
        pink_group_cyrillic.at[index, 'ORIGINAL JERSEY'] = 'Norbert Gyömbér'
        pink_group_cyrillic.at[index, 'Name(s) Found'] = 'Norbert Gyömbér'
    elif row['ORIGINAL JERSEY'] == '18. Уронен':
        pink_group_cyrillic.at[index, 'ORIGINAL JERSEY'] = 'Jere Uronen'
        pink_group_cyrillic.at[index, 'Name(s) Found'] = 'Jere Uronen'
    elif row['ORIGINAL JERSEY'] == '3. Гурман':
        pink_group_cyrillic.at[index, 'ORIGINAL JERSEY'] = 'Mark Gurman'
        pink_group_cyrillic.at[index, 'Name(s) Found'] = 'Mark Gurman'

        
        
        

        


In [359]:
for index, row in pink_group_cyrillic.iterrows():
    if row['ORIGINAL JERSEY'] == 'Norbert Gyömbér':
        pink_group_cyrillic.at[index, 'Name(s) Found'] = 'Norbert Gyomber'
        pink_group_cyrillic.at[index, 'ORIGINAL JERSEY'] = 'Norbert Gyomber'
    elif row['ORIGINAL JERSEY'] == 'Tome Pacovski':
        pink_group_cyrillic.at[index, 'Name(s) Found'] = 'Tome Pačovski'
        pink_group_cyrillic.at[index, 'ORIGINAL JERSEY'] = 'Tome Pačovski'

In [361]:
done_cases = 0
success_counter = 0
for index, row in pink_group_cyrillic.iterrows():
    if row['Status New'] != 'Done':
        name = row['Name(s) Found']
        full_num = row['Season']
        country_code_this = row['Country Code']
        dataset_nationality = leagues_value[(leagues_value['Team 1 Code'] == country_code_this) & 
                                    (leagues_value['Season'].isin([full_num, full_num + 1, full_num - 1]))]['Name'].unique()
        if name in dataset_nationality:
            correct_name = name
            
            print(index, name)

            number, reason = find_in_transfermarkt(correct_name, full_num, country_code_this, leagues_value)
            #print(player_this, reason, add_correct_money(number))

            #add a new row to lookup dataset 1 
            if type(number) == float:
                0==0
            else:
                number = add_correct_money(number)
            print('**')
            print(index, correct_name, number, reason)
            # if reason == 'Found in Curr Season':
            pink_group_cyrillic.at[index, 'Name(s) Found'] = correct_name
            pink_group_cyrillic.at[index, 'Status New'] = 'Done'
            pink_group_cyrillic.at[index, 'Market Value New'] = number
            pink_group_cyrillic.at[index, 'Match Case'] = 'single'
            pink_group_cyrillic.at[index, 'Lookup Return Case'] = reason
            success_counter += 1
        #print(index, row['ORIGINAL JERSEY'], row['Nationality'], row['Match'], row['Date'])
    else:
        done_cases += 1

print(success_counter, done_cases)

0 104


In [345]:
pink_group_cyrillic[pink_group_cyrillic['ORIGINAL JERSEY'] == '3. Гурман']

,DF index,Match ID,Competition,Date,Match,Season,Nationality,Country Code,ORIGINAL JERSEY,English Name,Name(s) Found,Status,Status New,Market Value New,Lookup Return Case,Match Case


In [363]:
pink_group_cyrillic.to_csv('pink_group_cyrillic_dudes_DONE.csv')

In [362]:
pink_group_cyrillic[pink_group_cyrillic['Status New'] != 'Done']['ORIGINAL JERSEY'].value_counts()

ORIGINAL JERSEY
5. Агиус Э.         2
10. Сливка          2
16. Алисами         2
2. Малый            2
3. Жута             2
15. Фере            2
18. Жута            2
17. Сынмэртян       2
13. Жоашим          2
16. Сливка          2
11. Алиев           2
22. Томсен Я.Л.     2
15. Хведукас        2
7. Чеснаускис Д.    1
8. Гыртымов         1
17. Петрович        1
3. Дембер           1
17.  Петрович       1
4. Петрович         1
5. Япасто           1
19. Кабрера Дж.     1
23. Шимич           1
6. Гусейнов Б.      1
2. К.Мартинс        1
9. Кибер            1
16. Фло             1
19. Кибер           1
3. Родич            1
20. Время           1
17. Оникэ           1
12. Бичер           1
13. С.Митрович      1
13. К.Оганесян      1
4. Н.Вукчевич       1
14. Нан Ваи Мин     1
15. Христов         1
7. Джавадов         1
22. Г.Илиев         1
4. Ф.Витайоли       1
11. Чезарини        1
5. Капилото         1
22. Хурме           1
3. Глостер          1
6. Хансон           1
4. Лукин        

## Edit Combined DF

In [189]:
combined_df = load_csv_dataset('combined_df_may_1.csv')
combined_df = combined_df.drop(columns={'Unnamed: 0'})

/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_48300/1791224083.py:49: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


In [191]:
tally = 0
for index, row in combined_df.iterrows():
    if combined_df.at[index, "Name(s) Found"].startswith('['):

        if len(eval(combined_df.at[index, "Name(s) Found"])) > 1:
            print(index, 'oh shit')
        else:
            tally += 1
            print(index, combined_df.at[index, "Name(s) Found"], eval(combined_df.at[index, "Name(s) Found"])[0])
            combined_df.at[index, "Name(s) Found"] = eval(combined_df.at[index, "Name(s) Found"])[0]
    elif type(combined_df.at[index, "Name(s) Found"]) == list:
        0==0
        #print(index, combined_df.at[index, "Name(s) Found"])
print(tally)

0
